# ИТ-платформа

In [ ]:
!pip install streamlit pyngrok

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# !pip install nest_asyncio

In [ ]:
!pip install mediapipe
!pip install ultralytics

  Using cached ultralytics-8.3.155-py3-none-any.whl.metadata (37 kB)
  Using cached ultralytics_thop-2.0.14-py3-none-any.whl.metadata (9.4 kB)
  Using cached nvidia_cuda_nvrtc_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-9.1.0.70-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.4.5.8-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.2.1.3-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.5.147-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.6.1.9-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.3.1.170-py3-none-manylinux2014_x86_64.whl.

In [ ]:
%%writefile AI_judging_RG.py
import streamlit as st
import cv2

import numpy as np
import pandas as pd
import os
import tempfile
import shutil
import time
import unicodedata
import re

from google.colab import files

from sklearn.preprocessing import LabelEncoder

import torch
import torch.nn as nn
import torchvision.transforms as T
import torchvision.models.video as models
from torchvision.transforms import Resize

import mediapipe as mp
from ultralytics import YOLO
from torchvision.models.segmentation import deeplabv3_resnet50

import plotly.express as px
import plotly.graph_objs as go
# import nest_asyncio

# nest_asyncio.apply()

# видео для тестирования
ELEMENTS_PATH = '/content/drive/MyDrive/диплом/По элементам'
# все промежуточные датасеты
DATASETS_PATH = '/content/drive/MyDrive/диплом/Datasets'
# веса модели
MODEL_WEIGHTS1 = '/content/drive/MyDrive/диплом/Datasets/r3d_18_weights/r3d_18_model_weights_4_epoch.pth'
MODEL_WEIGHTS2 = '/content/drive/MyDrive/диплом/Datasets/r3d_18_weights/r3d_18_model_weights_6_epoch.pth'
# deeplab weights
DEEPLAB_WEIGHTS = "/content/drive/MyDrive/диплом/data for Streamlit/deeplabv3_resnet50_coco-cd0a2569.pth"
# yolo weights
YOLO_WEIGHTS = "/content/drive/MyDrive/диплом/data for ai_judjig_rg/yolov8s.pt"

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') # переходим на гпу

### Модели предобработки видео
# YOLO
@st.cache_resource
def load_models():   # загрузка всех моделей
    """Загрузка моделей"""
    yolo_model = YOLO(YOLO_WEIGHTS) # загружаем веса предобученной модели
    deeplab_model = deeplabv3_resnet50(pretrained=False, weights_backbone=None)
    state_dict = torch.load(DEEPLAB_WEIGHTS, map_location=device)
    deeplab_model.load_state_dict(state_dict, strict=False)
    deeplab_model = deeplab_model.to(device).eval()
    return yolo_model, deeplab_model

# MediaPipe Pose
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(
    model_complexity=1,  #  сложность модели
    static_image_mode=False,  # обработка видеопотока, связывает порядок кадров
    min_detection_confidence=0.25,  # минимальный порог уверенности, что на видео есть человек
    min_tracking_confidence=0.4  # минимальный порог точек
)

# Нормализация для DeepLab
transform = T.Compose([
    T.ToPILImage(),   # перевод в изображение
    T.Resize((520, 520)),
    T.ToTensor(),  # в тензор
    T.Normalize(mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225])   # нормализация цветов
])

# Цвета для отрисовки частей тела
COLORS = {
    'left_arm': (0, 255, 0),  # зеленый
    'right_arm': (0, 200, 0),
    'left_leg': (0, 0, 255),  # красный
    'right_leg': (0, 0, 200),
    'torso': (255, 0, 0),   # синий
    'angle_hip_lines': (255, 255, 0),       # бирюзовый
    'angle_up90_lines': (255, 0, 255),      # сиреневый
    'angle_down90_lines': (255, 165, 0),    # желтый

}

# Части тела
LEFT_ARM = [mp_pose.PoseLandmark.LEFT_SHOULDER, mp_pose.PoseLandmark.LEFT_ELBOW, mp_pose.PoseLandmark.LEFT_WRIST]  # левая рука
RIGHT_ARM = [mp_pose.PoseLandmark.RIGHT_SHOULDER, mp_pose.PoseLandmark.RIGHT_ELBOW, mp_pose.PoseLandmark.RIGHT_WRIST]  # правая рука
LEFT_LEG = [mp_pose.PoseLandmark.LEFT_HIP, mp_pose.PoseLandmark.LEFT_KNEE, mp_pose.PoseLandmark.LEFT_ANKLE]  # левая нога
RIGHT_LEG = [mp_pose.PoseLandmark.RIGHT_HIP, mp_pose.PoseLandmark.RIGHT_KNEE, mp_pose.PoseLandmark.RIGHT_ANKLE] # правая нога
TORSO = [mp_pose.PoseLandmark.LEFT_SHOULDER, mp_pose.PoseLandmark.RIGHT_SHOULDER,
         mp_pose.PoseLandmark.RIGHT_HIP, mp_pose.PoseLandmark.LEFT_HIP]   # туловище

num_labels = 80  # количество элементов, на которых обучен датасет
df_labels = pd.read_excel(os.path.join(DATASETS_PATH, "LabelEncoder.xlsx"))

le = LabelEncoder()  # кодировка классов через labelencoder
df_labels['BD_label'] = le.fit_transform( df_labels['BD_names'])
# обратно из меток в названия классов для предикта
class_names = le.classes_

# нормализация символов
def normalize_text(text):
    # Нормализация Юникода (NFKD разбивает й → и + )
    text = unicodedata.normalize('NFKD', text)
    # Удаление диакритических символов
    text = ''.join(c for c in text if not unicodedata.combining(c))
    # Приведение к нижнему регистру
    text = text.lower()
    # Удаление лишних пробелов и замен на _
    text = re.sub(r'\s+', '_', text.strip())
    return text

# стоимость элементов
def read_element_base():
  cost_path = os.path.join(DATASETS_PATH, "Таблица трудностей тела.xlsx")
  df_balance = pd.read_excel(cost_path, sheet_name = 'РАВНОВЕСИЯ')
  df_balance['Название элемента'] = 'Равновесия_' + df_balance['Название элемента']
  df_turn = pd.read_excel(cost_path, sheet_name = 'ВРАЩЕНИЯ')
  df_turn['Название элемента'] = 'Повороты_' + df_turn['Название элемента']
  df_jumps = pd.read_excel(cost_path, sheet_name = 'ПРЫЖКИ')
  df_jumps['Название элемента'] = 'Прыжки_' + df_jumps['Название элемента']

  df_elements_base = pd.concat([df_balance,df_turn,df_jumps], axis = 0)
  df_elements_base['Название элемента'] = df_elements_base['Название элемента'].str.lower().str.strip()
  df_elements_base['Название элемента'] = df_elements_base['Название элемента'].apply(lambda x: normalize_text(x))

  return df_elements_base

# угол между 3 точками
def calculate_angle(a, b, c):
    """Считает угод между 3 точками"""
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)

    # Векторы BA и BC
    ba = a - b
    bc = c - b

    # Вычисляем угол между векторами через арктангенс от векторного и скалярного произведений
    cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
    sine_angle = np.cross(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))

    angle = np.arctan2(sine_angle, cosine_angle)  # [-pi, pi]
    angle = np.degrees(angle)
    if angle < 0:
        angle += 360  # Приводим к диапазону [0, 360)
    if angle > 210:
      angle = 360 - angle
    return angle

# Функция для предобработки видео
def processing_video(input_video_path, output_video_path):
    """Предобработка видео перед предиктом"""
    cap = cv2.VideoCapture(input_video_path)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)

    # VideoWriter с исходными размерами кадра
    # fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    fourcc = cv2.VideoWriter_fourcc(*'avc1')
    out = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))

    frame_numbers, seconds = [], []
    frames = []  # здесь будут уже уменьшенные тензоры: (C, H, W)
    original_frames = []  # оригинальный кадры в исходном размере
    processed_frames = []

    left_hips, right_hips = [], []
    left_knees, right_knees = [], []
    left_ankles, right_ankles = [], []
    left_shoulders, right_shoulders = [], []
    left_elbows, right_elbows = [], []
    left_wrists, right_wrists = [], []
    center_hips = []

    resize = Resize((112, 112))  # для ресайза к модели
    yolo_model, deeplab_model = load_models()

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = yolo_model.predict(frame, classes=[0], verbose=False)
        if results and results[0].boxes is not None and len(results[0].boxes) > 0:
            boxes = results[0].boxes.xyxy.cpu().numpy()
            bottoms = [y2 for x1, y1, x2, y2 in boxes]
            main_box = boxes[np.argmax(bottoms)]
            x1, y1, x2, y2 = map(int, main_box)
        else:
            x1, y1, x2, y2 = 0, 0, width, height

        cropped = frame_rgb[y1:y2, x1:x2]
        if cropped.size == 0:
            out.write(np.zeros_like(frame))  # Записываем черный кадр, если нет объекта
            continue

        input_tensor = transform(cropped).unsqueeze(0).to(device)
        with torch.no_grad():
            output = deeplab_model(input_tensor)['out'][0]
        output_predictions = output.argmax(0).byte().cpu().numpy()

        person_mask = (output_predictions == 15).astype(np.uint8) * 255
        person_mask = cv2.resize(person_mask, (x2 - x1, y2 - y1), interpolation=cv2.INTER_NEAREST)

        full_mask = np.zeros((height, width), dtype=np.uint8)
        full_mask[y1:y2, x1:x2] = person_mask

        black_frame = np.zeros_like(frame)
        silhouette_color = (180, 180, 180)
        silhouette = black_frame.copy()
        silhouette[full_mask == 255] = silhouette_color

        results_pose = pose.process(frame_rgb)
        if results_pose.pose_landmarks:
            landmarks = results_pose.pose_landmarks.landmark
            h, w, _ = frame.shape

            def get_point(lm): return int(landmarks[lm].x * w), int(landmarks[lm].y * h)

            for part, joints in [('left_arm', LEFT_ARM), ('right_arm', RIGHT_ARM),
                                 ('left_leg', LEFT_LEG), ('right_leg', RIGHT_LEG)]:
                for i in range(len(joints) - 1):
                    p1, p2 = get_point(joints[i]), get_point(joints[i + 1])
                    cv2.line(silhouette, p1, p2, COLORS[part], thickness=5)

            torso_points = np.array([get_point(p) for p in TORSO], dtype=np.int32)
            cv2.polylines(silhouette, [torso_points], isClosed=True, color=(255, 0, 0), thickness=3)

            for lm in mp_pose.PoseLandmark:
                x, y = get_point(lm)
                cv2.circle(silhouette, (x, y), 7, (255, 255, 255), -1)

            seconds.append(cap.get(cv2.CAP_PROP_POS_FRAMES) / fps)
            frame_numbers.append(cap.get(cv2.CAP_PROP_POS_FRAMES))

            # преобразование кадра в тензор
            silhouette_tensor = torch.from_numpy(silhouette.astype(np.float32) / 255.0).permute(2, 0, 1)
            silhouette_tensor = resize(silhouette_tensor)
            frames.append(silhouette_tensor)
            original_frames.append(frame.copy())  # сохраняем оригинальный кадр
            processed_frames.append(silhouette.copy()) # сохраняем обработанное видео

            # координаты
            # ноги
            left_hip, right_hip = get_point(mp_pose.PoseLandmark.LEFT_HIP), get_point(mp_pose.PoseLandmark.RIGHT_HIP)
            left_knee, right_knee = get_point(mp_pose.PoseLandmark.LEFT_KNEE), get_point(mp_pose.PoseLandmark.RIGHT_KNEE)
            left_ankle, right_ankle = get_point(mp_pose.PoseLandmark.LEFT_ANKLE), get_point(mp_pose.PoseLandmark.RIGHT_ANKLE)
            # руки
            left_shoulder, right_shoulder = get_point(mp_pose.PoseLandmark.LEFT_SHOULDER), get_point(mp_pose.PoseLandmark.RIGHT_SHOULDER)
            left_elbow, right_elbow = get_point(mp_pose.PoseLandmark.LEFT_ELBOW), get_point(mp_pose.PoseLandmark.RIGHT_ELBOW)
            left_wrist, right_wrist = get_point(mp_pose.PoseLandmark.LEFT_WRIST), get_point(mp_pose.PoseLandmark.RIGHT_WRIST)
            # точка между левым и правым бедром
            center_hip = ((left_hip[0] + right_hip[0]) // 2, (left_hip[1] + right_hip[1]) // 2)

            left_hips.append(left_hip)
            right_hips.append(right_hip)
            center_hips.append(center_hip)
            left_knees.append(left_knee)
            right_knees.append(right_knee)
            left_ankles.append(left_ankle)
            right_ankles.append(right_ankle)
            left_shoulders.append(left_shoulder)
            right_shoulders.append(right_shoulder)
            left_elbows.append(left_elbow)
            right_elbows.append(right_elbow)
            left_wrists.append(left_wrist)
            right_wrists.append(right_wrist)

        # Сохраняем кадр в исходном разрешении
        # out.write(silhouette)  # Записываем кадр с силуэтом в оригинальном размере

    df_coordinates = pd.DataFrame({
        'Frame number': frame_numbers,
        'Time, second': seconds,
        'Left shoulder': left_shoulders,
        'Right shoulder': right_shoulders,
        'Left Elbow': left_elbows,
        'Right Elbow': right_elbows,
        'Left wrist': left_wrists,
        'Right wrist': right_wrists,
        'Left hip': left_hips,
        'Right hip': right_hips,
        'Center hip': center_hips,
        'Left knee': left_knees,
        'Right knee': right_knees,
        'Left ankle': left_ankles,
        'Right ankle': right_ankles
    })

    # Сборка итогового тензора: (1, C, T, H, W)
    frames = torch.stack(frames, dim=1).unsqueeze(0)

    cap.release()
    out.release()

    return frames, df_coordinates, fps, original_frames, processed_frames

class VideoClassifier(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.resnet = models.r3d_18(pretrained = False)
        self.resnet.fc = nn.Linear(self.resnet.fc.in_features, num_classes)

    def forward(self, x):
        return self.resnet(x)

model1 = VideoClassifier(num_classes = num_labels)  # модель, предикт 80 классов
model1.load_state_dict(torch.load(MODEL_WEIGHTS1, map_location=device))  # загружаем сохраненные веса модели
model1.to(device)

model2 = VideoClassifier(num_classes = num_labels)  # модель, предикт 80 классов
model2.load_state_dict(torch.load(MODEL_WEIGHTS2, map_location=device))  # загружаем сохраненные веса модели
model2.to(device)

def predict(frames, model):
    """Возвращает классы для всех элементов в видео"""
    model.eval()
    with torch.no_grad():
        output = model(frames.to(device))
        probs = torch.softmax(output, dim=1)
        prob, class_ = torch.max(probs, dim=1)

        predicted_class = torch.argmax(output, dim=1).item()
    class_name = le.inverse_transform([predicted_class])[0]
    df_elements_base = read_element_base()
    norm_class_name = normalize_text(class_name.lower().strip())

    matched_rows = df_elements_base[df_elements_base['Название элемента'] == norm_class_name]

    if matched_rows.empty:
        item_cost = 0.0
        # print(f"Элемент не найден в базе: '{norm_class_name}'")
    else:
        item_cost = matched_rows['Стоимость'].values[0]
        #     print("Предсказанный класс:", class_name)
    return class_name, item_cost, prob.item()

# угол между бедрами (шпагат)
def calculate_hip_angle(right_knee,center_hip,left_knee):
  if right_knee[0] >= left_knee[0]: # правая нога правее левой
    legs_angle =  360 - calculate_angle(left_knee, center_hip,right_knee)
  if right_knee[0]< left_knee[0]: # правая нога левее левой (стоит левым боком)
    legs_angle =  calculate_angle(  left_knee,center_hip, right_knee)
  return legs_angle

# угол в колене (проверка, что колено прямое или наоборот согнутое)
def calculate_knee_angle(left_hip, left_knee, left_ankle, right_hip, right_knee, right_ankle):
  # считаем угол в колене на той ноге, у которой выше стопа по координате y
  if right_ankle[1] > left_ankle[1]:
    hip,knee, ankle = right_hip, right_knee, right_ankle
  else:
    hip,knee, ankle = left_hip, left_knee, left_ankle   # если левая стопа выше, считаем угол в левом колене
  # угол в колене
  legs_angle = calculate_angle(ankle, knee, hip)
  return legs_angle

 # угол в бедре относительно плеча
def calculate_straight_angle(knee, hip, shoulder):
  angle =  calculate_angle(knee, hip, shoulder)
  return angle

# углы между бедрами; плечом, бедром и коленом; в колене
# элементы, в которых нужен шпагат
twine_180_elements = [
    'Повороты_Арабеск',
    'Повороты_Аттитюд',
    'Повороты_Боковой шпагат без помощи, туловище в горизонтальном положении',
    'Повороты_Боковой шпагат с помощью',
    'Повороты_Боковой шпагат с помощью, туловище в горизонтальном положении',
    'Повороты_В шпагате с помощью с наклоном вперед',
    'Повороты_В шпагате с помощью с наклоном назад',
    'Повороты_Вперед: свободная нога в горизонтальном положении',
    'Повороты_Задний шпагат без помощи, туловище горизонтально',
    'Повороты_Задний шпагат без помощи, туловище горизонтально в кольцо',
    'Повороты_Задний шпагат с помощью',
    'Повороты_Кольцо без помощи аттитюд с прогибом назад',
    'Повороты_Кольцо с помощью с ногой на плече',
    'Повороты_На животе груди, ноги в положении подбива с помощью («Ашрам»)',
    'Повороты_На животе груди, ноги в положении шпагата («Канаева»)',
    'Повороты_Передний шпагат без помощи',
    'Повороты_Передний шпагат с наклоном туловища назад в горизонталь',
    'Повороты_Передний шпагат с помощью',
    'Повороты_Передний шпагат, с наклоном туловища назад ниже горизонтали',
    'Повороты_Фуэте боковой шпагат с помощью',
    'Повороты_Фуэте: пассе',
    'Повороты_Фуэте: прямая нога в горизонтальном положении',
    'Повороты_Циркуль назад',

    'Прыжки_«Казак» – нога в сторону вверх, вся стопа выше головы, без помощи',
    'Прыжки_«Перекидной» («Entrelace»)',
    'Прыжки_«Перекидной» («Entrelace») с прогибом',
    'Прыжки_«Фуете»',
    'Прыжки_«Фуете» подбивной с прогибом',
    'Прыжки_«Фуете» с подбивом',
    'Прыжки_«Фуете» с прогибом',
    'Прыжки_В боковой  шпагат',
    'Прыжки_В шпагат',
    'Прыжки_В шпагат в кольцо ',
    'Прыжки_В шпагат в кольцо с поворотом',
    'Прыжки_В шпагат с поворотом',
    'Прыжки_В шпагат с поворотом со сменой ног',
    'Прыжки_В шпагат с прогибом',
    'Прыжки_В шпагат с прогибом с поворотом',
    'Прыжки_В шпагат со сменой прямых ног',
    'Прыжки_В шпагат со сменой прямых ног с прогибом',
    'Прыжки_Зарипова(ZR)',
    'Прыжки_Подбивной',
    'Прыжки_Подбивной в кольцо',
    'Прыжки_Подбивной в кольцо с поворотом',
    'Прыжки_Подбивной с поворотом',
    'Прыжки_Подбивной с поворотом со сменой ног',
    'Прыжки_Подбивной с поворотом со сменой ног с прогибом',
    'Прыжки_Подбивной с прогибом',
    'Прыжки_Подбивной с прогибом с поворотом',
    'Прыжки_Подбивной с прогибом со сменой ног',
    'Прыжки_Подбивной со сменой ног',

    'Равновесия_Арабеск',
    'Равновесия_Арабеск с прогибом назад',
    'Равновесия_Арабеск с туловищем вперед в горизонтальном положении',
    'Равновесия_Аттитюд', 'Равновесия_Аттитюд с наклоном туловища назад Кольцо без помощи',

    'Равновесия_Боковой шпагат без помощи рук',
    'Равновесия_Боковой шпагат без помощи, туловище в сторону в горизонталь',
    'Равновесия_Боковой шпагат с помощью рук',
    'Равновесия_Задний шпагат с помощью',
    'Равновесия_Задний шпагат с помощью наклон туловища вперед в горизонталь',
    'Равновесия_Задний шпагат с помощью наклон туловища вперед в горизонталь в кольцо',
    'Равновесия_Задний шпагат с помощью наклон туловища вперед в горизонталь с помощью',
    'Равновесия_Итальянское фуете',
    'Равновесия_Кольцо с помощью',
    'Равновесия_Пассе',
    'Равновесия_Передний шпагат без помощи рук',
    'Равновесия_Передний шпагат без помощи рук, наклон туловища назад в горизонталь',
    'Равновесия_Передний шпагат без помощи, наклон туловища назад ниже горизонтали',
    'Равновесия_Передний шпагат с помощью рук',
    'Равновесия_Свободная нога горизонтально вперед прямая',
    'Равновесия_Утяшева'
]

# элементы, в которых нужен угол 90* между верхним плечом бедром и коленом
straight_up_90_elements = [
    'Повороты_Арабеск',
    'Повороты_Аттитюд',
    'Повороты_Боковой шпагат без помощи, туловище в горизонтальном положении',
    'Повороты_Боковой шпагат с помощью, туловище в горизонтальном положении',
    'Повороты_Вперед: свободная нога в горизонтальном положении',
    'Повороты_Задний шпагат без помощи, туловище горизонтально',
    'Повороты_Задний шпагат без помощи, туловище горизонтально в кольцо',
    'Повороты_Кольцо без помощи аттитюд с прогибом назад',
    'Повороты_Пассе',
    'Повороты_Передний шпагат с наклоном туловища назад в горизонталь',
    'Повороты_Фуэте: пассе',
    'Повороты_Фуэте: прямая нога в горизонтальном положении',

    'Равновесия_Арабеск',
    'Равновесия_Аттитюд',
    'Равновесия_Боковой шпагат без помощи, туловище в сторону в горизонталь',
    'Равновесия_Задний шпагат с помощью наклон туловища вперед в горизонталь',
    'Равновесия_Задний шпагат с помощью наклон туловища вперед в горизонталь с помощью',
    'Равновесия_Пассе',
    'Равновесия_Передний шпагат без помощи рук, наклон туловища назад в горизонталь',
    'Равновесия_Свободная нога горизонтально вперед прямая',
    ]

# элементы, в которых нужен угол 90* между нижним плечом бедром и коленом
straight_down_90_elements = [
    'Повороты_Боковой шпагат без помощи, туловище в горизонтальном положении',
    'Повороты_Боковой шпагат с помощью, туловище в горизонтальном положении',
    'Повороты_Задний шпагат без помощи, туловище горизонтально',
    'Повороты_Задний шпагат без помощи, туловище горизонтально в кольцо',
    'Повороты_Передний шпагат с наклоном туловища назад в горизонталь',
    'Равновесия_Арабеск с туловищем вперед в горизонтальном положении',
    'Равновесия_Боковой шпагат без помощи, туловище в сторону в горизонталь',
    'Равновесия_Задний шпагат с помощью наклон туловища вперед в горизонталь',
    'Равновесия_Задний шпагат с помощью наклон туловища вперед в горизонталь с помощью',
    'Равновесия_Передний шпагат без помощи рук, наклон туловища назад в горизонталь',
]

# отрисовка углов
def draw_angle(current_shot, BD_name,current_frames):   # координаты, класс элемента
  # display(current_shot)
  center_hip = current_shot['Center hip'].values[0]
  left_hip = current_shot['Left hip'].values[0]
  right_hip = current_shot['Right hip'].values[0]
  left_knee = current_shot['Left knee'].values[0]
  right_knee =   current_shot['Right knee'].values[0]
  left_shoulder = current_shot['Left shoulder'].values[0]
  right_shoulder =   current_shot['Right shoulder'].values[0]
  left_ankle = current_shot['Left ankle'].values[0]
  right_ankle =   current_shot['Right ankle'].values[0]

  # рисуем поверх угол
  # угол между бедрами в шпагат
  if BD_name in twine_180_elements:

    cv2.line(current_frames, center_hip,left_knee, COLORS['angle_hip_lines'], thickness=2)
    cv2.line(current_frames,  center_hip, right_knee, COLORS['angle_hip_lines'], thickness=2)
    # Подпись угла
    cv2.putText(current_frames, f'{int(current_shot["Hip angle"].values[0])} deg',
              (center_hip[0] - 50, center_hip[1] - 20),
              cv2.FONT_HERSHEY_SIMPLEX, 1, COLORS['angle_hip_lines'], 2, cv2.LINE_AA)

  if BD_name in straight_up_90_elements:   # угол 90* между верхним плечом бедром и коленом

    if left_ankle[1] < right_ankle[1]:   # левая нога выше правой
      knee, hip, shoulder = left_knee, left_hip, left_shoulder
    else:  # правая нога сверху
      knee, hip, shoulder = right_knee, right_hip, right_shoulder

    cv2.line(current_frames, shoulder, hip, COLORS['angle_up90_lines'], thickness=2)  # линия плечо бедро
    cv2.line(current_frames,  hip, knee, COLORS['angle_up90_lines'], thickness=2)  # линия бедро колено
    # Подпись угла
    cv2.putText(current_frames, f'{int(current_shot["Up straight angle"].values[0])} deg',
              (hip[0] + 50, hip[1] + 20),
              cv2.FONT_HERSHEY_SIMPLEX, 1, COLORS['angle_up90_lines'], 2, cv2.LINE_AA)

  if BD_name in straight_down_90_elements:   # угол 90* между нижним плечом бедром и коленом

    if left_ankle[1] < right_ankle[1]:   # левая нога выше правой
      knee, hip, shoulder = right_knee, right_hip, right_shoulder  # нижняя нога правая
    else:  # нижняя нога левая
      knee, hip, shoulder = left_knee, left_hip, left_shoulder

    cv2.line(current_frames, shoulder, hip, COLORS['angle_down90_lines'], thickness=2)  # линия плечо бедро
    cv2.line(current_frames,  hip, knee, COLORS['angle_down90_lines'], thickness=2)  # линия бедро колено
    # Подпись угла
    cv2.putText(current_frames, f'{int(current_shot["Down straight angle"].values[0])} deg',
              (hip[0] - 100, hip[1] - 40),
              cv2.FONT_HERSHEY_SIMPLEX, 1,COLORS['angle_down90_lines'], 2, cv2.LINE_AA)

  return current_frames

def segment_and_classify(frames_tensor, df_coordinates, fps,
                       processed_frames, original_frames, output_dir,
                       window_size=1.5, step=1.0, threshold=0.80
                       ):

    """Поиск и классификация элементов в видео"""
    # Если папка с видео уже есть, то удаляем ее и создаем заново с пустым содержанием
    if os.path.exists(output_dir):
        print("Directory exists")
        shutil.rmtree(output_dir)
        os.mkdir(output_dir)
    else:
        print("Doesn't exists")
        os.mkdir(output_dir)
        if os.path.exists(output_dir):
            print("Directory successful created")

    predictions = []
    coords = []

    total_frames = frames_tensor.shape[2]
    duration = total_frames / fps

    if duration < window_size:
        window_size = duration

    window_frames = int(window_size * fps)
    step_frames = int(step * fps)
    current_time = 0
    clip_count = 0

    while current_time + window_size <= duration:
        start_frame = int(current_time * fps)
        end_frame = start_frame + window_frames

        if end_frame > total_frames:
            break

        segment_tensor = frames_tensor[:, :, start_frame:end_frame, :, :]
        prediction1, item_cost1, prob1 = predict(segment_tensor, model1)
        prediction2, item_cost2, prob2 = predict(segment_tensor, model2)

        if prob1 > prob2:
            prob, prediction, item_cost = prob1, prediction1, item_cost1
        else:
            prob, prediction, item_cost = prob2, prediction2, item_cost2

        if prob >= threshold:
            # Сохраняем координаты
            save_coords = df_coordinates[
                (df_coordinates['Frame number'] >= start_frame) &
                (df_coordinates['Frame number'] <= end_frame)
            ].copy()
            save_coords['Time, duration'] = save_coords['Time, second'] - save_coords['Time, second'].min()

            # Создаем безопасное имя файла
            safe_name = re.sub(r'[\\/*?:"<>|]', "", prediction)
            safe_name = safe_name.replace(" ", "_")
            filename = f"{safe_name}_{clip_count:02d}.mp4"
            clip_count += 1

            # Обработанное видео
            processed_segment = []
            for i in range(start_frame, end_frame):
                if i < len(processed_frames):
                    frame = processed_frames[i]
                    if i in df_coordinates['Frame number'].astype(int).values:
                        current_shot = df_coordinates[df_coordinates['Frame number'] == i]
                        frame = draw_angle(current_shot, prediction, frame)
                    processed_segment.append(frame)

            # Оригинальное видео
            original_segment = original_frames[start_frame:min(end_frame, len(original_frames))]

            # Сохраняем видео в байты
            video_bytes_orig = frames_to_bytes(original_segment, fps)
            video_bytes_proc = frames_to_bytes(processed_segment, fps/4)

            predictions.append({
                "DB": prediction,
                "Стоимость": item_cost,
                "Видеофрагмент элемента": video_bytes_orig,
                "Силуэт элемента": video_bytes_proc,
                "Вероятность принадлежности к классу": f"{round(prob * 100, 2)}%"
            })

            coords.append(save_coords)

        current_time += step

    return pd.DataFrame(predictions), coords

def frames_to_bytes(frames, fps):
    if not frames or len(frames) == 0:
        st.warning("No frames to convert!")
        return b''

    # Конвертация кадров в RGB
    frames = [frame if len(frame.shape) == 3 and frame.shape[2] == 3
             else cv2.cvtColor(frame, cv2.COLOR_GRAY2BGR)
             for frame in frames]

    frames = [frame.astype('uint8') for frame in frames]

    try:
        h, w = frames[0].shape[:2]
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        with tempfile.NamedTemporaryFile(suffix='.mp4', delete=False) as tmp_file:
            temp_path = tmp_file.name

        out = cv2.VideoWriter(temp_path, fourcc, max(1, int(fps)), (w, h))
        if not out.isOpened():
            st.error("Failed to open VideoWriter")
            return b''

        for frame in frames:
            out.write(frame)
        out.release()

        with open(temp_path, 'rb') as f:
            return f.read()
    except Exception as e:
        st.error(f"Video writing error: {str(e)}")
        return b''
    finally:
        if os.path.exists(temp_path):
            os.unlink(temp_path)

def evaluation_criteria():
    """Стоимости и критерии оценивания для конкретных элементов"""
    pass

# # Оформление платформы
st.title("Поддержка системы судейства в художественной гимнастике")
video_file = st.file_uploader("Загрузите видео",
                              type=["mp4", "avi", "mov"]
                             )

if video_file is not None:
    st.success("Видео успешно загружено!")
    st.video(video_file)
    tfile = tempfile.NamedTemporaryFile(delete=False)
    tfile.write(video_file.read())
    input_path = tfile.name
    output_path = input_path.replace(".mp4", "_processed.mp4")

    # Временный текст, который заменится на результат модели при выполнении всех функций
    temp_text = st.empty()
    with st.spinner("Выполняется обработка видео и оценивание..."):

        frames, df_coordinates, fps, original_frames, processed_frames = processing_video(input_path, output_path)

        df_coordinates['Hip angle'] = df_coordinates.apply(lambda x: calculate_hip_angle(x['Right knee'], x['Center hip'], x['Left knee']), axis = 1)
        df_coordinates['Knee angle'] = df_coordinates.apply(lambda x: calculate_knee_angle(x['Left hip'], x['Left knee'], x['Left ankle'],x['Right hip'], x['Right knee'], x['Right ankle']), axis = 1)

        # Меньшее значение Y — выше
        df_coordinates['Up straight angle'] = df_coordinates.apply(
            lambda x: calculate_straight_angle(x['Left knee'], x['Left hip'], x['Left shoulder'])
            if x['Left ankle'][1] < x['Right ankle'][1]  # инвертировано
            else calculate_straight_angle(x['Right knee'], x['Right hip'], x['Right shoulder']), axis=1)

        df_coordinates['Down straight angle'] = df_coordinates.apply(
            lambda x: calculate_straight_angle(x['Left knee'], x['Left hip'], x['Left shoulder'])
            if x['Left ankle'][1] > x['Right ankle'][1]  # инвертировано
            else calculate_straight_angle(x['Right knee'], x['Right hip'], x['Right shoulder']), axis=1)

        df_results, coords = segment_and_classify(frames, df_coordinates, fps,
                                                  processed_frames, original_frames,
                                                  output_dir="/content/drive/MyDrive/диплом/proc videos",
                                                  window_size=2, step=1, threshold=0.80
                                                  )

        total_value = round(df_results['Стоимость'].sum(), 1)

        hip_angles = []
        for i in range(len(coords)):
            hip_angles.append(list(coords[i]['Hip angle']))

        df_results['Hip angle'] = hip_angles

        st.dataframe(df_results[["DB", "Стоимость", "Hip angle"]],
                     column_config={
                            "DB": "Элемент DB",
                            "Стоимость": "Стоимость",
                            "Hip angle": st.column_config.LineChartColumn(
                                "Угол между бедрами",
                                y_min = 0, y_max = 270
                            ),
                            }
                        )


  #     evaluation = evaluation_criteria()
        temp_text.empty()
        st.metric("Итоговая стоимость DB:", value = total_value)
        st.divider()
        st.subheader("Подробный разбор видео")
        st.divider()

        for num_row in range(df_results.shape[0]):
            st.write(f'Элемент {num_row+1}: ')

            orig, proc = st.columns(2)

            orig_video_path = df_results.loc[num_row, 'Видеофрагмент элемента']
            proc_video_path = df_results.loc[num_row, 'Силуэт элемента']

            name_orig_video = os.path.basename(orig_video_path)
            name_proc_video = os.path.basename(proc_video_path)

            def display_video(video_path, column):
                if video_path and os.path.exists(video_path):
                    try:
                        with open(video_path, "rb") as f:
                            video_bytes = f.read()
                            column.video(video_bytes, format="video/mp4")
                    except Exception as e:
                        column.error(f"Ошибка загрузки видео: {str(e)}")
                else:
                    column.warning(f"Файл не найден: {video_path}")


            display_video(orig_video_path, orig)
            display_video(proc_video_path, proc)



            # if os.path.exists(orig_video_path):
            #     tmp_orig = tempfile.NamedTemporaryFile(delete=False, suffix=".mp4")
            #     shutil.copy(orig_video_path, tmp_orig.name)
            #     tmp_orig.close()
            #     orig.video(tmp_orig.name)
            # else:
            #     orig.warning("Оригинальное видео не найдено.")

            # # Обработанное видео
            # if os.path.exists(proc_video_path):
            #     tmp_proc = tempfile.NamedTemporaryFile(delete=False, suffix=".mp4")
            #     shutil.copy(proc_video_path, tmp_proc.name)
            #     tmp_proc.close()
            #     proc.video(tmp_proc.name)
            # else:
            #     proc.warning("Обработанное видео не найдено.")

Overwriting AI_judging_RG.py


In [ ]:
!ngrok authtoken '2wpQQHvfvkeGxIwStgV7VCojdBh_ZGRSc9uiUmav9x87DEVV'

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
from pyngrok import ngrok

# Открываем туннель на порту 8501
public_url = ngrok.connect(8501)
print('Streamlit app is live at:', public_url)

# Запуск Streamlit
!streamlit run AI_judging_RG.py &

Streamlit app is live at: NgrokTunnel: "https://9686-34-34-55-98.ngrok-free.app" -> "http://localhost:8501"



  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.34.55.98:8501

2025-06-15 18:35:33.902 Uncaught exception GET /_stcore/stream (127.0.0.1)
HTTPServerRequest(protocol='http', host='9686-34-34-55-98.ngrok-free.app', method='GET', uri='/_stcore/stream', version='HTTP/1.1', remote_ip='127.0.0.1')
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/streamlit/web/bootstrap.py", line 347, in run
    if asyncio.get_running_loop().is_running():
       ^^^^^^^^^^^^^^^^^^^^^^^^^^
RuntimeError: no running event loop

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/tornado/websocket.py", line 938, in _accept_connection
    open_result = handler.open(*h

Test

In [ ]:
input_video_path = '/content/Киря_Яблочникова_мяч_17_сек_—_сделано_в_Clipchamp.mp4'
output_video_path = input_video_path.replace(".mp4", "_processed.mp4")

frames, df_coordinates, fps, original_frames,processed_frames = processing_video(input_video_path, output_video_path)

2025-05-10 19:59:05.739 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-10 19:59:06.156 
  command:

    streamlit run /usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py [ARGUMENTS]
2025-05-10 19:59:06.159 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-10 19:59:06.160 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-10 19:59:06.661 Thread 'Thread-9': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-10 19:59:06.663 Thread 'Thread-9': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-10 19:59:08.908 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-10 19:59:08.909 Thread 'MainThread': missin

In [ ]:
df_coordinates['Hip angle'] = df_coordinates.apply(lambda x: calculate_hip_angle(x['Right knee'], x['Center hip'], x['Left knee']), axis = 1)
df_coordinates['Knee angle'] = df_coordinates.apply(lambda x: calculate_knee_angle(x['Left hip'], x['Left knee'], x['Left ankle'],x['Right hip'], x['Right knee'], x['Right ankle']), axis = 1)
# Меньшее значение Y — выше
df_coordinates['Up straight angle'] = df_coordinates.apply(
    lambda x: calculate_straight_angle(x['Left knee'], x['Left hip'], x['Left shoulder'])
    if x['Left ankle'][1] < x['Right ankle'][1]  # инвертировано
    else calculate_straight_angle(x['Right knee'], x['Right hip'], x['Right shoulder']), axis=1)

df_coordinates['Down straight angle'] = df_coordinates.apply(
    lambda x: calculate_straight_angle(x['Left knee'], x['Left hip'], x['Left shoulder'])
    if x['Left ankle'][1] > x['Right ankle'][1]  # инвертировано
    else calculate_straight_angle(x['Right knee'], x['Right hip'], x['Right shoulder']), axis=1)
df_coordinates.head(3)

,Frame number,"Time, second",Left shoulder,Right shoulder,Left Elbow,Right Elbow,Left wrist,Right wrist,Left hip,Right hip,Center hip,Left knee,Right knee,Left ankle,Right ankle,Hip angle,Knee angle,Up straight angle,Down straight angle
0,1.0,0.033333,"(738, 402)","(737, 405)","(800, 484)","(790, 489)","(855, 560)","(841, 555)","(784, 571)","(766, 576)","(775, 573)","(835, 715)","(789, 729)","(873, 862)","(772, 877)",17.777579,164.898369,184.276021,178.923829
1,2.0,0.066667,"(738, 402)","(740, 407)","(800, 482)","(796, 489)","(855, 557)","(844, 556)","(784, 571)","(768, 576)","(776, 573)","(835, 714)","(789, 728)","(874, 862)","(772, 876)",17.912139,165.581377,184.402010,178.458733
2,3.0,0.100000,"(740, 402)","(747, 406)","(804, 478)","(809, 485)","(860, 550)","(855, 553)","(788, 571)","(775, 576)","(781, 573)","(840, 715)","(791, 729)","(882, 862)","(779, 875)",18.894752,169.331304,183.999413,176.617037


In [ ]:
df_results, coords = segment_and_classify(frames, df_coordinates, fps,
                         processed_frames, original_frames,
                         window_size=2, step=1, threshold=0.80,
                         output_dir="classified_clips001")
df_results

<ipython-input-5-156a3d63fedd>:594: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  save_coords['Time, duration'] = save_coords['Time, second'] -  save_coords['Time, second'].min()
<ipython-input-5-156a3d63fedd>:594: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  save_coords['Time, duration'] = save_coords['Time, second'] -  save_coords['Time, second'].min()
<ipython-input-5-156a3d63fedd>:594: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer

,DB,Стоимость,Видеофрагмент элемента,Силуэт элемента,Вероятность принадлежности к классу
0,Равновесия_Задний шпагат с помощью наклон тул...,0.5,classified_clips001/original/Равновесия_Заднии...,classified_clips001/processed/Равновесия_Задни...,99.79%
1,Равновесия_Задний шпагат с помощью наклон тул...,0.5,classified_clips001/original/Равновесия_Заднии...,classified_clips001/processed/Равновесия_Задни...,100.0%
2,Равновесия_Задний шпагат с помощью наклон тул...,0.5,classified_clips001/original/Равновесия_Заднии...,classified_clips001/processed/Равновесия_Задни...,82.71%
3,"Повороты_Задний шпагат без помощи, туловище г...",0.5,classified_clips001/original/Повороты_Задний ...,classified_clips001/processed/Повороты_Задний...,96.6%
4,"Повороты_Задний шпагат без помощи, туловище г...",0.5,classified_clips001/original/Повороты_Задний ...,classified_clips001/processed/Повороты_Задний...,100.0%
5,"Повороты_Задний шпагат без помощи, туловище г...",0.5,classified_clips001/original/Повороты_Задний ...,classified_clips001/processed/Повороты_Задний...,99.7%


In [ ]:
df_results.loc[0, 'Силуэт элемента']

'classified_clips001/processed/Равновесия_Задний шпагат с помощью наклон туловища вперед в горизонталь в кольцо_01.mp4'

In [ ]:
import os

path = 'classified_clips001/processed/Равновесия_Задний шпагат с помощью наклон туловища вперед в горизонталь в кольцо_01.mp4'
print(os.path.exists(path))

True


In [ ]:
hip_angles = []
for i in range(len(coords)):
    hip_angles.append(list(coords[i]['Hip angle']))
df_results['Hip angle'] = hip_angles

In [ ]:
for num_row in range(df_results.shape[0]):
    st.write(f'Элемент {num_row+1}: ')
    orig, proc = st.columns(2)
    orig_video_path = df_results.loc[num_row, 'Видеофрагмент элемента']
    proc_video_path = df_results.loc[num_row, 'Силуэт элемента']
    if os.path.exists(orig_video_path):
        with open(orig_video_path, "rb") as f:
            orig_video_bytes = f.read()
            orig.video(orig_video_bytes)
    # else:
    #     orig.warning("Оригинальное видео не найдено.")
    # Проверка и вывод обработанного видео
    if os.path.exists(proc_video_path):
        with open(proc_video_path, "rb") as f:
            proc_video_bytes = f.read()
            proc.video(proc_video_bytes)
    # else:
    #     proc.warning("Обработанное видео не найдено.")

2025-05-10 20:04:45.524 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-10 20:04:45.526 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-10 20:04:45.526 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-10 20:04:45.528 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-10 20:04:45.529 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-10 20:04:45.531 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-10 20:04:45.531 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-10 20:04:45.532 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

In [ ]:
orig_video_path

'classified_clips001/original/Повороты_Задний шпагат без помощи, туловище горизонтально_06.mp4'

In [ ]:
orig

DeltaGenerator()

In [ ]:
st.video(input_video_path)

2025-05-10 20:05:59.031 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-10 20:05:59.037 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-10 20:05:59.039 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


DeltaGenerator()
